In [1]:
from swin_model import Swin_model
from unet_model import Unet_model
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 
import monai
import numpy as np
from monai.inferers import sliding_window_inference

In [2]:
def load_model():
    swin_8687 = Swin_model("/scratch/scratch6/akansh12/challenges/parse2022/temp/selected_models/swin-again_no_back_1000hu_8655.pth")
    swin_8675 = Swin_model("/scratch/scratch6/akansh12/challenges/parse2022/temp/selected_models/swin-again_no_back_1000hu_8675.pth")
    swin_8655 = Swin_model("/scratch/scratch6/akansh12/challenges/parse2022/temp/selected_models/swin-again_no_back_1000hu_8687.pth")
    unet_8530 = Unet_model("/scratch/scratch6/akansh12/challenges/parse2022/temp/selected_models/unet_1000_hu_160_0853.pth")
    unet_8550 = Unet_model("/scratch/scratch6/akansh12/challenges/parse2022/temp/selected_models/unet_1000_hu_160_8550.pth")
    unet_8551 = Unet_model("/scratch/scratch6/akansh12/challenges/parse2022/temp/selected_models/unet_1000_hu_160_w_augmentations_8551.pth")
    
    return swin_8687, swin_8675, swin_8655, unet_8530, unet_8550, unet_8551

In [3]:
swin_8687, swin_8675, swin_8655, unet_8530, unet_8550, unet_8551 = load_model()
ensemble_weights = [88, 87, 86.55, 85.30, 85.50, 85.51]

### Dataset

In [4]:
import glob
import os
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd, 
    EnsureTyped,
    EnsureType,
    Invertd,
    KeepLargestConnectedComponent,
    AddChanneld,
    ToTensord

)
test_transforms = Compose(
    [
        LoadImaged(keys=["images"]),
        EnsureChannelFirstd(keys=["images"]),
        Orientationd(keys=["images"], axcodes="LPS"),
        ScaleIntensityRanged(
            keys=["images"], a_min=-1000, a_max=1000,
            b_min=0.0, b_max=1.0, clip=True,
        ),
        CropForegroundd(keys=["images"], source_key="images"),
        EnsureTyped(keys=["images"]),
    ]
)

def test_dataloader(path2input, test_transforms = test_transforms):
    root_dir = path2input
    test_files_path = sorted(glob.glob(os.path.join(root_dir, "**/*.nii.gz"), recursive = True))
    test_data = [{"images": image_name } for image_name in test_files_path]
    test_ds = Dataset(data = test_data, transform=test_transforms)
    test_loader = DataLoader(test_ds, batch_size = 1, shuffle = False)
    return test_loader

In [5]:
loader = test_dataloader("/scratch/scratch6/akansh12/challenges/parse2022/docker/test_inputs/")

In [6]:
post_transforms_1 = Compose([AsDiscreted(keys = "pred", argmax=True)])
post_transforms_2 = Compose([
        Invertd(
            keys="pred",
            transform=test_transforms,
            orig_keys="images",
            meta_keys=None,
            orig_meta_keys=None,
            meta_key_postfix="meta_dict",
            nearest_interp=False,
            to_tensor=True,
        ),
        SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir="/scratch/scratch6/akansh12/challenges/parse2022/docker/test_outputs/", output_postfix='seg', resample=False),
    ])

In [7]:
models = [unet_8530, unet_8550, unet_8551, swin_8687, swin_8675, swin_8655]
weights = np.array([85.30, 85.50, 85.51, 88, 87, 86.55])
weights = weights / np.sum(weights)

In [8]:
roi_size = (288, 288, 288)
sw_batch_size = 8
with torch.no_grad():
    for test_data in loader:
        test_data_out = 0
        for e, (model,w) in enumerate(zip(models,weights)):
            out = {}
            test_inputs = test_data["images"].to(device)
            model = model.to(device)
            if e > 2:
                roi_size = (96, 96, 96)
                sw_batch_size = 4
            out["pred"] = sliding_window_inference(test_inputs, roi_size, sw_batch_size, model)
            out = [post_transforms_1(i) for i in decollate_batch(out)]
            test_data_out = test_data_out + w*out[0]['pred'][0]
            torch.cuda.empty_cache()
            
        test_data_out = test_data_out > 0.5
        test_data_out = test_data_out.unsqueeze(0).unsqueeze(0)
        test_data['pred'] = test_data_out
        test_data = [post_transforms(i) for i in decollate_batch(test_data)]
        

None of the inputs have requires_grad=True. Gradients will be None


NameError: name 'post_transforms' is not defined

In [10]:
def predict_and_save(loader, model, path2save,test_transforms, roi_size = (288, 288, 288), sw_batch_size = 8):
    os.makedirs(path2save, exist_ok=True)
    post_transforms = Compose([
        Invertd(
            keys="pred",
            transform=test_transforms,
            orig_keys="images",
            meta_keys=None,
            orig_meta_keys=None,
            meta_key_postfix="meta_dict",
            nearest_interp=False,
            to_tensor=True,
        ),
        AsDiscreted(keys="pred", argmax=True),
        SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir=path2save, output_postfix='seg', resample=False),
    ])   
    
    model.eval()
    with torch.no_grad():
        for test_data in loader:
            test_inputs = test_data["images"].to(device)
            test_data["pred"] = sliding_window_inference(test_inputs, roi_size, sw_batch_size, model)
            test_data = [post_transforms(i) for i in decollate_batch(test_data)]
    torch.cuda.empty_cache()


In [ ]:
## Unet
predict_and_save(loader, unet_8530, "./test_outputs/unet_8530_out/", test_transforms)
predict_and_save(loader, unet_8550, "./test_outputs/unet_8550_out/", test_transforms)
predict_and_save(loader, unet_8551, "./test_outputs/unet_8551_out/", test_transforms)

#Swin
predict_and_save(loader, swin_8655, "./test_outputs/swin_8655_out/", test_transforms, roi_size = (96, 96, 96), sw_batch_size = 4)
predict_and_save(loader, swin_8675, "./test_outputs/swin_8675_out/", test_transforms, roi_size = (96, 96, 96), sw_batch_size = 4)
predict_and_save(loader, swin_8687, "./test_outputs/swin_8687_out/", test_transforms, roi_size = (96, 96, 96), sw_batch_size = 4)



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.9 µs


None of the inputs have requires_grad=True. Gradients will be None


2022-08-02 13:33:13,097 INFO image_writer.py:190 - writing: test_outputs/swin_8655_out/PA000005/PA000005_seg.nii.gz
2022-08-02 13:46:40,340 INFO image_writer.py:190 - writing: test_outputs/swin_8655_out/PA000016/PA000016_seg.nii.gz
2022-08-02 14:00:06,800 INFO image_writer.py:190 - writing: test_outputs/swin_8675_out/PA000005/PA000005_seg.nii.gz
2022-08-02 14:13:27,728 INFO image_writer.py:190 - writing: test_outputs/swin_8675_out/PA000016/PA000016_seg.nii.gz
2022-08-02 14:26:53,304 INFO image_writer.py:190 - writing: test_outputs/swin_8687_out/PA000005/PA000005_seg.nii.gz


### Ensemble

In [29]:
subject_names = []

In [30]:
for i in glob.glob("./test_outputs/**/*.nii.gz", recursive=True):
    subject_names.append(i.split('/')[-1])

In [34]:
np.unique(subject_names)[0]

'PA000005_seg.nii.gz'

In [39]:
sorted(glob.glob(f"./test_outputs/**/{np.unique(subject_names)[0]}", recursive=True))

['./test_outputs/swin_8655_out/PA000005/PA000005_seg.nii.gz',
 './test_outputs/swin_8675_out/PA000005/PA000005_seg.nii.gz',
 './test_outputs/swin_8687_out/PA000005/PA000005_seg.nii.gz',
 './test_outputs/unet_8551_out/PA000005/PA000005_seg.nii.gz']

In [12]:
for x in loader:
    break

In [13]:
x

{'images': tensor([[[[[0.0040, 0.0005, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0010, 0.0020, 0.0035,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0010, 0.0000,  ..., 0.0000, 0.0100, 0.0075],
            ...,
            [0.0000, 0.0000, 0.0010,  ..., 0.0000, 0.0000, 0.0000],
            [0.0050, 0.0000, 0.0010,  ..., 0.0000, 0.0035, 0.0075],
            [0.0010, 0.0035, 0.0010,  ..., 0.0100, 0.0070, 0.0080]],
 
           [[0.0050, 0.0000, 0.0000,  ..., 0.0015, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
            [0.0000, 0.0000, 0.0035,  ..., 0.0000, 0.0000, 0.0000],
            ...,
            [0.0000, 0.0000, 0.0035,  ..., 0.0000, 0.0055, 0.0060],
            [0.0005, 0.0000, 0.0000,  ..., 0.0010, 0.0060, 0.0035],
            [0.0000, 0.0000, 0.0000,  ..., 0.0065, 0.0070, 0.0005]],
 
           [[0.0065, 0.0000, 0.0000,  ..., 0.0035, 0.0020, 0.0010],
            [0.0070, 0.0000, 0.0000,  ..., 0.0030, 0.0000, 0.0000]